This notebook is to test the functionalities without having to run the whole script repeatedly.

It automatically extracts the contents of a Word document, according to the given format, saves the extracted information into dictionary variables, which are pickled and stored locally.

In [2]:
from docx.text.paragraph import Paragraph
from docx.document import Document
from docx.table import _Cell, Table
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
import docx
from openai import OpenAI
import pickle

iter_bock_items() is the function used to iterate over paragraphs and tables in a docx document

In [14]:
def iter_block_items(parent):
    if isinstance(parent, Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            #yield Paragraph(child, parent)
            current_p=Paragraph(child,parent)
            yield current_p
        elif isinstance(child, CT_Tbl):
            table = Table(child, parent)
            for row in table.rows:
                for cell in row.cells:
                    yield from iter_block_items(cell)

Read the desired Word document
(Can't run if the document is open)

In [24]:
doc = docx.Document('Projectreports/Projektreport 2023_V02_Beispiel.docx')

Initialise Dictionaries that will store the information from the Word document (RN: untill chapter 5: Macht und Interessen)

Infos that do not belong to any Chapter:

In [17]:
#Erste Tabelle Titelseite
D_Titelseite={'Projekt':'',
              'Teilnehmer':'',
              'Firma':'',
              'Adresse':'',
              'Telefon':'',
              'E-Mail':'',
              'Kurs':''}
#D_Projektname={'Projektname':''}

#Änderungshistorie Tabelle
D_Änderungshistorie={'1-Version':'',
                     '1-Datum':'',
                     '1-Ersteller':'',
                     '1-Grund':''} #Further entries are added later on 

#I name this entries chapter 0
D_Chapt_0={'Angaben zum Projekt':'',
           'Eigene Position':''}




Infos from chapt 1. Strategie 4.5.1.

In [18]:
D_Strategie={'Beschreibung des Business Case zum Projekt':'',
                 'Nennung der kritischen Erfolgsfaktoren des Projekts':''}

Infos from chapt 2. Governance, Strukturen und Prozesse 04.03.02

In [19]:
D_Governance_Strukturen_Prozesse={'Begründung, warum ist es ein Projekt':'',
                                  'Projektart und Begründung':'',
                                  'Klassufizierung aus Sicht der Organisation':'',
                                  'Strukturen der Organisation':''}

Infos from chapt 3. Anforderungen und Ziele 4.5.2.

In [20]:
D_Steckbrief={'Projektbezeichnung':'', 
              'Projektnummer':'',
              'Auftraggeber':'',
              'Projektleitung, Verantwortung':'',
              'Leistung':None,
              'Termine':None,
              'Kosten':None,
              'Projektorganisationsform':'',
              'Stabs-PO':None,
              'Matrix-PO':None,
              'Reine_PO':None,
              'Oberziel':'',
              'Projektinhalt':'',
              'Projektbeteiligte':'',
              'Projektumfeld':'',
              'Starttermin':'',
              'Endetermin': '',
              'Dauer': '',
              'Zwischentermine':'',
              'Aufwand-gesamt':'',
              'Aufwand-intern':'',
              'Aufwand-extern':'',
              'Personalkosten-gesamt':'',
              'Personalkosten-intern':'',
              'Personalkosten-extern':'',
              'Sachkosten-gesamt':'',
              'Sachkosten-intern':'',
              'Sachkosten-extern':'',
              'Investitionen-gesamt':'',
              'Investitionen-intern':'',
              'Investitionen-extern':'',
              'Budget-gesamt':'',
              'Budget-intern':'',
              'Budget-extern':'',
              'vrsl. Behinderungen/Risiken/Störungen':'',
              'Kunde':'',
              'Abnahmekriterien':''}

D_Oberziel={'Bezeichnung':'', 'Beschreibung':'', 'Messkriterium':''}

#Ergebnisziele:
D_Finanzziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Leistungsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}        #Further entries are added later on 
D_Qualitätsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}        #Further entries are added later on 
D_SozialeZiele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}          #Further entries are added later on 

#Vorgehensziele:
D_Terminziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Kostenziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Aufwandsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}         #Further entries are added later on 
D_Projektrahmenziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}    #Further entries are added later on 

#Nicht-Ziele:
D_NichtZiele={'1-Bezeichnung':'', '1-Beschreibung':''}                                  #Further entries are added later on 

#Zielkonflikte
D_Zielkonflikte={'1-Ziel_a':'',
                 '1-Ziel_b':'',
                 '1-Konfliktart':'',
                 '1-Priorität':'',
                 '1-Erklärung':'',
                 '1-Massnahmen':''} #Further entries are added later on 

Infos from chapt 4. Stakeholder 4.5.12.

In [21]:
D_Projektumfeld={'sozial-intern':'',
                 'sozial-extern':'',
                 'sachlich-intern':'',
                 'sachlich-extern':''}

D_Schnittstellen_Stakeholder_Umfeld={'1-Schnittstelle':'',
                                     '1-Ansprechpartner':'',
                                     '1-Auswirkung':'',
                                     '2-Schnittstelle':'',
                                     '2-Ansprechpartner':'',
                                     '2-Auswirkung':''} #Further entries may be added later on 

D_Stakeholderinteressen={'1-Stakeholder':'',
                         '1-Konfliktpotentzial':'',
                         '1-Einfluss/Macht':'',
                         '1-Erwartungen/Befrüchtungen':'',
                         '1-Strategie':'',
                         '1-Steuerung/Massnahmen':''} #Further entries are added later on 

Infos in chapt. 5. Macht und Interessen 04.03.04

In [22]:
#Maybe fuse into one dictionary
D_Stakeholderbewertung={'1-Stakeholder':'',
                        '1-Macht':'',
                        '1-Begründung der Machtposition':''} #Further entries are added later on 

D_Machtpromotoren={'Machtpromotoren':''}

Prints all text blocks

In [23]:
past_block=''
for block in iter_block_items(doc):
    current_block=block.text
    # ignore repeated cells, #ignore paragraphs starting with ignore_list
    if current_block != past_block: #and not current_block.startswith(ignore_list[0]) and not current_block.startswith(ignore_list[1]) and not current_block.startswith(ignore_list[2]) and not current_block.startswith(ignore_list[3]) and not current_block.startswith(ignore_list[4])
        print(block.text)
        #print(block.style)
    past_block=current_block

Hinweis:
Dieser Beispiel-Report dient dazu, sich Ideen zu möglichen Inhalten für Kapitel zu holen. Es ist keine Kopiervorlage!
Es ist kein „perfekter“ Report. Den kann es aufgrund von unterschiedlichen Sichtweisen der Assessoren auch gar nicht geben.
Wenn es kleine Abweichungen zwischen dem aktuellen Report-Template und diesem Beispiel gibt, orientiert euch bitte am Template.

Report Level D

nach Z01D_Leitfaden / 02 vom 20.06.2023

Dieses Dokument basiert auf der Report-Vorlage der Decisio Projekt- und Prozessmanagement GmbH.

Projekt:
Android-App „Prioritize“
Teilnehmer:
Max Mustermanager
Firma:
alltagsApps GmbH
Adresse:
Musterweg 11, 11111 Atwork
Telefon:
0170-12345678
E-Mail:
MaxMustermanager@alltagsApps.de
Kurs:
23-1999

Änderungshistorie	
Version
Datum
Ersteller
Grund
0.1
02.08.23
Max Mustermanager
Ersterstellung
0.2
06.08.23
Max Mustermanager
1. Überarbeitung
0.3
15.08.23
Max Mustermanager
2. Überarbeitung
1.0
28.08.23
Max Mustermanager
Fertigstellung

Inhaltsverzeichnis
1.	Stra